In [1]:
import pandas as pd

import sys
sys.path.insert(1, '../../../../scripts/')
from s3_support import *

# removing dupes

In [5]:
df_priorpreds = get_dataframe_from_file("qgiv-stats-data", "preds.churn.csv")

In [3]:
df_priorpreds['date_predicted'].unique()[-1]

'2021-03-01 14:37:41.802758'

In [4]:
df_priorpreds = df_priorpreds[df_priorpreds['date_predicted']!=df_priorpreds['date_predicted'].unique()[-1]]
df_priorpreds['date_predicted'].unique()

array([], dtype=object)

In [5]:
#save_dataframe_to_file("qgiv-stats-data", "preds.churn.csv", df_priorpreds)

uploading to S3
Done


# spot checking

In [25]:
df = get_dataframe_from_file("qgiv-stats-data", "preds.churn.csv")

In [9]:
df['date_predicted'] = pd.to_datetime(df['date_predicted'])
df.tail(3)

,date_predicted,org
7,2021-03-01 14:37:41.802758,1323
8,2021-03-01 14:37:41.802758,445963
9,2021-03-01 14:37:41.802758,1486


In [10]:
df[df['date_predicted']=='2021-02-02']

,date_predicted,org


In [26]:
df

,Date,ID,Org Name
0,2021-03-08 15:57:28.999971,444911,NaN
1,2021-03-08 15:57:28.999971,441903,NaN
2,2021-03-08 15:57:28.999971,331779,NaN
3,2021-03-08 15:57:28.999971,444083,NaN
4,2021-03-08 15:57:28.999971,442539,NaN
5,2021-03-08 15:57:28.999971,497,NaN
6,2021-03-08 15:57:28.999971,444329,NaN
7,2021-03-08 15:57:28.999971,1323,NaN
8,2021-03-08 15:57:28.999971,1486,NaN
9,2021-03-08 15:57:28.999971,441576,NaN


In [14]:
df_growth = get_dataframe_from_file("qgiv-stats-data", "preds.growth.csv")

In [16]:
len(df_growth), df_growth['date'].unique()

(20, array(['2021-03-01 14:39:16.450798'], dtype=object))

# restoring histories

In [3]:
churn = pd.read_csv("preds.churn.csv")
growth = pd.read_csv("preds.growth.csv")

In [8]:
churn.head(3)

,Date,Org Name
0,6/10/2019,Capital Roots
1,6/17/2019,"Fort Lauderdale Area Youth for Christ, Inc."
2,6/17/2019,Friends of Heather Cordasco


In [4]:
growth.head(3)

,Date,Org Name
0,9/10/2019,Oliver Gospel Mission (Top 100)
1,9/10/2019,The For a Day Foundation
2,9/10/2019,Acenda


In [5]:
org_names = get_dataframe_from_file("qgiv-stats-data", "organizations.names.csv")

In [7]:
org_names[['id', 'org_name']].head(3)

,id,org_name
0,441789,NaN
1,443044,NaN
2,1045,FROM Food Fight


In [13]:
def get_org_name(name):
    try:
        return org_names[org_names['org_name']==name]['id'].iloc[0]
    except:
        return 0

growth['ID'] = growth['Org Name'].apply(get_org_name).astype(int)
churn['ID'] = churn['Org Name'].apply(get_org_name).astype(int)

In [19]:
print("no churn ID: {} ({:.1f}%)".format(len(churn[churn['ID']==0]), (len(churn[churn['ID']==0]) / len(churn)) * 100.))
print("no growth ID: {} ({:.1f}%)".format(len(growth[growth['ID']==0]), (len(growth[growth['ID']==0]) / len(growth)) * 100.))

no churn ID: 91 (17.0%)
no growth ID: 184 (22.8%)


In [21]:
growth[growth['Org Name'].str.contains('\(')]

,Date,Org Name,ID
0,9/10/2019,Oliver Gospel Mission (Top 100),0
92,2/17/2020,Colorado Family Life Center (CFLC),29686
143,3/9/2020,Urban Roots (DBA for Austin Youth & Community,0
180,3/23/2020,PCAF (Pierce County AIDS Foundation),1215
593,8/24/2020,Court Appointed Special Advocate (CASA),0


In [23]:
org_names[org_names['org_name'].str.contains('Urban Roots').fillna(False)]

,id,org_name,tax_id,sic,csm,street,city,state,zip,country,...,status,pricing_package,reseller,previously_accepted_credit_online,segment,tags,date_closed,reason_closed,new_provider,additional_churn_info
8857,7089,Urban Roots (DBA for Austin Youth & Community ...,453954705,8398.0,heidi.morris@qgiv.com,4900 Gonzales St.,Austin,Texas,78702,US,...,active,Package Pricing,Qgiv,No,O - Youth Development,"Tier 3 - ARR < $4,500",NaN,NaN,NaN,NaN


In [24]:
churn.to_csv("preds.churn.csv", index=False)
growth.to_csv("preds.growth.csv", index=False)